##Context based LLM Fine-tuning

In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 2

In [3]:
import json
with open(r"/content/train.json", "r") as read_file:
    train = json.load(read_file)

In [4]:
train

[{'context': 'A normal menstrual cycle typically ranges from 21 to 35 days, with the average cycle lasting around 28 days.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'What is a normal menstrual cycle length?',
    'answers': [{'text': '21 to 35 days', 'answer_start': 71}]}]},
 {'context': 'Common causes of irregular periods include hormonal imbalances, stress, excessive exercise, weight changes, thyroid disorders, and certain medical conditions.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'What are common causes of irregular periods?',
    'answers': [{'text': 'hormonal imbalances, stress, excessive exercise, weight changes, thyroid disorders, and certain medical conditions',
      'answer_start': 28}]}]},
 {'context': 'Menstrual cramps can be alleviated through various methods including over-the-counter pain relievers, applying heat to the abdomen, gentle exercise, relaxation techniques, and dietary changes.',
  'qas': [{'id': '

In [5]:
with open(r"/content/test.json", "r") as read_file:
    test = json.load(read_file)

In [6]:
test

[{'context': 'Challenges associated with Menstrual Hygiene Management include lack of access to affordable menstrual products, inadequate sanitation facilities, cultural taboos and stigmas, and limited knowledge about menstrual hygiene.',
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'What are some challenges associated with Menstrual Hygiene ?',
    'answers': [{'text': 'lack of access to affordable menstrual products, inadequate sanitation facilities, cultural taboos and stigmas, and limited knowledge about menstrual hygiene',
      'answer_start': 38}]}]},
 {'context': 'Menstrual Hygiene Management (MHM) refers to the practices and facilities needed to ensure that menstruating individuals can manage their menstruation with dignity, including access to menstrual products, proper sanitation facilities, and education about menstrual hygiene.',
  'qas': [{'id': '00002',
    'is_impossible': False,
    'question': 'What is Menstrual Hygiene Management (MHM)?',
   

In [7]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [8]:

model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"


In [9]:

# Configure the model
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs= 20



In [10]:
pip install wandb

In [11]:

### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 20,
    "evaluate_during_training_steps": 1000,
    # "wandb_project": "Question Answer Application",
    # "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [12]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [13]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args, use_cuda=False
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [14]:
# Train the model
model.train_model(train, eval_data=test)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
convert squad examples to features:   0%|          | 0/12 [00:00<?, ?it/s]Could not find answer: 'average cycle' vs. '21 to 35 days'
Could not find answer: 'periods include hormonal imbalances, stress, excessive exercise, weight changes, thyroid disorders, and certain medical' vs. 'hormonal imbalances, stress, excessive exercise, weight changes, thyroid disorders, and certain medical conditions'
Could not find answer: 'pads or tampons every hour for several consecutive hours, passing large blood clots, and experiencing symptoms like fatigue and shortness of breath.' vs. 'soaking through one or more pads or tampons every hour for several consecutive hours, passing large blood clots, and experiencing symptoms like fatigue and shortness of breath'
Could not fin

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 1 of 20:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

convert squad examples to features: 100%|██████████| 10/10 [00:00<00:00, 231.96it/s]

add example index and unique id: 100%|██████████| 10/10 [00:00<00:00, 58908.76it/s]


Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 19 of 20:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 20 of 20:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(20,
 {'global_step': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20],
  'correct': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'similar': [5, 5, 4, 5, 5, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7],
  'incorrect': [5, 5, 6, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
  'train_loss': [4.872088432312012,
   4.899232387542725,
   4.482168197631836,
   3.765293598175049,
   3.05413818359375,
   2.401066303253174,
   1.726556420326233,
   1.3072636127471924,
   0.9529024362564087,
   0.6713464856147766,
   0.6220480799674988,
   0.49333474040031433,
   0.454741895198822,
   0.3662862777709961,
   0.24918417632579803,
   0.164272740483284,
   0.16991247236728668,
   0.16041199862957,
   0.16260293126106262,
   0.11050257086753845],
  'eval_loss': [-0.045486100018024445,
   -0.0533997118473053,
   -0.07763777673244476,
   -0.11669416725635529,
   -0.16495643556118011,
   -0.218956351

In [15]:
# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
print(result)

{'correct': 0, 'similar': 7, 'incorrect': 3, 'eval_loss': -0.7606507539749146}


In [17]:
print(texts)

{'correct_text': {}, 'similar_text': {'00001': {'truth': 'lack of access to affordable menstrual products, inadequate sanitation facilities, cultural taboos and stigmas, and limited knowledge about menstrual hygiene', 'predicted': 'lack of access to affordable menstrual products, inadequate sanitation facilities, cultural taboos and stigma', 'question': 'What are some challenges associated with Menstrual Hygiene ?'}, '00003': {'truth': 'in accordance with local waste management guidelines and according to local waste management guidelines', 'predicted': 'according to local waste management', 'question': 'How should menstrual products be properly disposed of?'}, '00004': {'truth': 'eusable, environmentally friendly, cost-effective over time, and can be worn for longer periods', 'predicted': 'usable, environmental', 'question': 'What are the advantages of using menstrual cups?'}, '00006': {'truth': 'dysmenorrhea (menstrual cramps), menorrhagia (heavy menstrual bleeding), amenorrhea (abse

In [18]:


# Make predictions with the model
to_predict = [
    {
        "context": "menstruation may vary to around a 4 to 8 days. for some it can be short but others bit longer",
        "qas": [
            {
                "question": "How long does menstruation last?",
                "id": "00006",
            }
        ],
    }
]


In [19]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 8176.03it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '00006', 'answer': ['4 to 8', '4', '4 to 8 days. for some it can be short but others bit longer']}]
